In [12]:
#Author: Mihai Avram, e-mail: mihai.v.avram@gmail.com

#for parsing logs
import json
#for connecting to the database
import psycopg2

#function for deciding on a score value to use for bot_score_english and bot_score_universal depending on what's available in the log
def score_decider(potential_score_keys, line_json):
    for key in potential_score_keys:
        if len(key) == 1:
            key1 = key[0]
            try:
                score = line_json[key1]
                print("proper key found")
                return score
            except:
                continue
        elif len(key) == 2:
            key1 = key[0]
            key2 = key[1]
            try:
                score = line_json[key1][key2]
                print("proper key found")
                return score
            except:
                continue
    
    return None


#connecting to the database
pgsqlconn = psycopg2.connect(host='localhost', user='postgres', password='SMajinboos2@', dbname='botbase')
#cursor needed to execute db operations
botbase_cursor = pgsqlconn.cursor()

def sql_execute(user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, \
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, \
                num_mentions, latest_tweet_timestamp, num_requests, user_profile):
    
    # cur.execute("""INSERT INTO botscore (an_int, a_date, a_string) VALUES (%s, %s, %s);""", \
    #             (10, datetime.date(2005, 11, 18), "O'Reilly"))
    botbase_cursor.execute("""INSERT INTO public.botscore(
                user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, 
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, 
                num_mentions, latest_tweet_timestamp, num_requests, user_profile) 
                              VALUES 
                (%s, %s, to_timestamp(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", \
                (user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, \
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, \
                num_mentions, latest_tweet_timestamp, num_requests, user_profile))

    #commiting changes
    pgsqlconn.commit()

#parsing logs and uploading the entries to the botometer database
logFile = open("log-2.txt","r")

step = 0
for line in logFile:
    if step == 1:
        break
    line_json = json.loads(line)
    print("NEW ENTRY")
    user_id = line_json["search"]["user_id"]
    print("user_id:",line_json["search"]["user_id"])
    screen_name = line_json["search"]["sn"]
    print("screen_name:",line_json["search"]["sn"])
    time_stamp = line_json["timestamp"]
    print("time_stamp:",line_json["timestamp"])
    #categories = line_json["categories"]
    print("categories:",line_json["categories"])
    #english bot score which is either found in line_json["score"], line_json["classification"], line_json["score"]["english"]
    keys = [["score","english"],["score"],["classification"]]
    bot_score_english = score_decider(keys, line_json)
    print("bot_score_english:", bot_score_english)
    #universal bot score which is either found in line_json["score"]["universal"] or line_json["categories"]["languageagnostic_classification"] otherwise null
    keys = [["score","universal"],["categories","languageagnostic_classification"]]
    bot_score_universal = score_decider(keys, line_json)
    print("bot_score_universal:", bot_score_universal)
    requester_ip = line_json["remote_ip"]
    print("requester_ip:",str(line_json["remote_ip"]))
    tweets_per_day = None
    print("tweets_per_day:", "NULL")
    num_tweets = line_json["num_tweets"]
    print("num_tweets:", line_json["num_tweets"])
    num_mentions = None
    print("num_mentions:","NULL")
    latest_tweet_timestamp = None
    print("latest_tweet_timestamp:","NULL")
    num_requests = 1
    print("num_requests:", "lookup previous + 1")
    user_profile = None
    print("user_profile:","NULL")
    sql_execute(user_id, screen_name, time_stamp, None, bot_score_english, bot_score_universal, \
                requester_ip, tweets_per_day, num_tweets, num_mentions, latest_tweet_timestamp, num_requests, user_profile)
    step = step + 1
    
#closing access to database
botbase_cursor.close()
pgsqlconn.close()

#closing log file
logFile.close()



NEW ENTRY
user_id: 428845131
screen_name: awesomely_hk
time_stamp: 1435698355
categories: {'user_classification': 0.18, 'content_classification': 0.23, 'sentiment_classification': 0.46, 'friend_classification': 0.1, 'network_classification': 0.09, 'temporal_classification': 0.16}
proper key found
bot_score_english: 0.21
bot_score_universal: None
requester_ip: ['172.29.33.45', '137.132.3.8']
tweets_per_day: NULL
num_tweets: 200
num_mentions: NULL
latest_tweet_timestamp: NULL
num_requests: lookup previous + 1
user_profile: NULL
